In [2]:
import pandas as pd

In [14]:
orders_columns = [
    'order_id',
    'order_date',
    'order_customer_id',
    'order_status'
]

In [15]:
orders = pd.read_csv('../data/retail_db/orders/part-00000', header=None, names=orders_columns)

Tasks related to aggregations

- get count by order_status
- get count by order month and then order status. We need to generate order month using order data
  using apply on dataframe

orders.apply(lambda order: order_date[:7], axis =1)

In [5]:
orders.columns

Index(['order_id', 'order_date', 'order_customer_id', 'order_status'], dtype='object')

In [7]:
orders.groupby?

Signature:
orders.groupby(
    by=None,
    axis: 'Axis | lib.NoDefault' = <no_default>,
    level: 'IndexLabel | None' = None,
    as_index: 'bool' = True,
    sort: 'bool' = True,
    group_keys: 'bool' = True,
    observed: 'bool | lib.NoDefault' = <no_default>,
    dropna: 'bool' = True,
) -> 'DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, pd.Grouper or list of such
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If a list or ndarray of le

In [16]:
orders. \
    groupby('order_status')['order_id']. \
    agg('count')

order_status
CANCELED            1428
CLOSED              7556
COMPLETE           22899
ON_HOLD             3798
PAYMENT_REVIEW       729
PENDING             7610
PENDING_PAYMENT    15030
PROCESSING          8275
SUSPECTED_FRAUD     1558
Name: order_id, dtype: int64

In [17]:
# .apply() is a low level api where we can specify a lambda function (to obtain the order_month)
orders['order_month'] = orders.apply(lambda order: order.order_date[:7], axis=1)

In [20]:
orders

,order_id,order_date,order_customer_id,order_status,order_month
0,1,2013-07-25 00:00:00.0,11599,CLOSED,2013-07
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT,2013-07
2,3,2013-07-25 00:00:00.0,12111,COMPLETE,2013-07
3,4,2013-07-25 00:00:00.0,8827,CLOSED,2013-07
4,5,2013-07-25 00:00:00.0,11318,COMPLETE,2013-07
...,...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE,2014-07
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE,2014-07
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT,2014-07
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD,2014-07


In [22]:
orders. \
    groupby(['order_month','order_status'])['order_id']. \
    agg(order_count='count')

order_count
order_month order_status                
2013-07     CANCELED                  22
            CLOSED                   161
            COMPLETE                 515
            ON_HOLD                   81
            PAYMENT_REVIEW            19
...                                  ...
2014-07     PAYMENT_REVIEW            54
            PENDING                  517
            PENDING_PAYMENT          979
            PROCESSING               561
            SUSPECTED_FRAUD          101

[117 rows x 1 columns]

In this case, order_month and order_status are considered as keys, not actual values.
We use reset_index() to convert them to 'values'

In [23]:
orders. \
    groupby(['order_month','order_status'])['order_id']. \
    agg(order_count='count') .\
    reset_index()

,order_month,order_status,order_count
0,2013-07,CANCELED,22
1,2013-07,CLOSED,161
2,2013-07,COMPLETE,515
3,2013-07,ON_HOLD,81
4,2013-07,PAYMENT_REVIEW,19
...,...,...,...
112,2014-07,PAYMENT_REVIEW,54
113,2014-07,PENDING,517
114,2014-07,PENDING_PAYMENT,979
115,2014-07,PROCESSING,561
